In [3]:
import cv2
import yt_dlp
import time
from datetime import datetime
from ultralytics import solutions
from ultralytics import YOLO
from collections import defaultdict

import pandas as pd
import pickle

#model = YOLO("yolo11n.pt")
#model = YOLO("yolo11m.pt")

In [4]:
import sys
sys.version

'3.10.16 | packaged by Anaconda, Inc. | (main, Dec 11 2024, 16:19:12) [MSC v.1929 64 bit (AMD64)]'

In [5]:
def get_stream_url(youtube_url):
    ydl_opts = {
        'quiet': True,
        'skip_download': True,
        'no_warnings': True,
        'force_generic_extractor': False,
        'format': 'best[ext=mp4][protocol^=http]/best'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(youtube_url, download=False)
        formats = info.get('formats', [info])
        # prefer video-only best mp4 OR best overall
        for f in reversed(formats):
            if f.get('vcodec', '') != 'none' and f.get('acodec', '') == 'none' and 'url' in f:
                return f['url']
        # fallback: best available format
        return formats[-1]['url']

In [10]:
def cap_video_output(input_info, 
                     limit_sec = 120,    # how long to record the video
                     frame_interval = 5,   # record nth frame based on the frame_interval
                     show_frames = True   # show the video being recorded
                    ):   

    
    # get video URL feed
    video_url = input_info["url"]
    stream_url = get_stream_url(video_url)
    #print("Stream URL:", stream_url)

    cap = cv2.VideoCapture(stream_url)
    assert cap.isOpened(), "Error reading video file"    

    print("Input: ", input_info)

    # use current time as part of output file name
    current_time = time.time()
    datetime_object = datetime.fromtimestamp(current_time)
    datetime_string = datetime_object.strftime("%y%m%d.%H%M")        
    out_filename = input_info["filename"]+"."+ datetime_string
    filetype = input_info["filetype"]
    out_filename_all = out_filename + "." + filetype
    print("out_filename_all", out_filename_all)
    
    # Video writer
    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(out_filename_all, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))    
    
    
    # Start timer
    start_time = time.time()
    prev_elapsed = 0
    #limit_sec = 120

    frame_no = 0
    new_frame_ctr = 0
    #frame_interval = 6  # if frame_interval = 5 we will capture every 5th frame
    timestamps = []
    timestamps_d = defaultdict(int)  # dictionary object to store timestamps

    print("Working: ", end="")
    while cap.isOpened():
        success, im0 = cap.read()

        if not success:
            print("Video frame is empty or processing is complete.")
            break

        if frame_no % frame_interval == 0:
            video_writer.write(im0)  # write the original frame
            # The original input frame is shown in the window 
            if show_frames:
                cv2.imshow('Original', im0)   # show the original frame
            timestamp_ms = cap.get(cv2.CAP_PROP_POS_MSEC)  # this is timestamp starting at 0
            current_time = time.time()
            datetime_object = datetime.fromtimestamp(current_time)
            #datetime_string = datetime_object.strftime("%Y-%m-%d %H:%M:%S")
            datetime_string = datetime_object.strftime("%F %T.%f")[:-3]
            timestamps.append((frame_no, current_time, datetime_string))  # store current time
            timestamps_d[new_frame_ctr] = current_time
            new_frame_ctr +=1 # increment our new frame counter
        frame_no += 1
    
        # Check elapsed time if greater than limit, break
        current_time = time.time()
        elapsed_time = current_time - start_time
        
        
        # print asterisks every min to indicate progress
        curr_elapsed = round(elapsed_time)        
        if (round(elapsed_time)%60 == 0 and curr_elapsed > prev_elapsed):
            print("*", end="")
        
        if (round(elapsed_time)%3600 == 0 and curr_elapsed > prev_elapsed):
            print(round(elapsed_time/3600,0), "HR")
        prev_elapsed = curr_elapsed
        
        #print(elapsed_time)
        if elapsed_time > limit_sec: 
            break

        # Exit loop if 'q' key is pressed - often this requires multiple presses of q key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("manually quitting")
            break

    print("")
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()  # destroy all opened windows

    current_time = time.time()
    datetime_object = datetime.fromtimestamp(current_time)
    datetime_string = datetime_object.strftime("%F %T.%f")[:-3]    
    
    
    print(datetime_string, " video capture and output file done")
    
    
    # convert to dataframe and save to pickle file for later processing if needed
    ts = pd.DataFrame(timestamps) 
    pickle_file = out_filename + ".pkl"
    ts.to_pickle(pickle_file)
    print("Created pickle file: ", pickle_file)
    
    #return timestamps_d  # return timestamp for each frame (dictionary)
    return out_filename, filetype, ts  # return name of file, filetpe and timestamps list as DataFrame

In [8]:
# define input parameters
input_info = {
    "filename" : "BridgeAmerNorth",
    "filetype" : "mp4",
    "url": "https://www.youtube.com/watch?v=mp3RS0y77tY"
}
print(input_info)
print(input_info["url"])

{'filename': 'BridgeAmerNorth', 'filetype': 'mp4', 'url': 'https://www.youtube.com/watch?v=mp3RS0y77tY'}
https://www.youtube.com/watch?v=mp3RS0y77tY


In [11]:
# CAPTURE CAMERA FEED VIDEO
output_file, output_filetype, ts = cap_video_output(input_info)

#output_file, output_filetype, ts = cap_video_output(input_info, limit_sec=30, frame_interval=5)
print("output_file", output_file, output_filetype)

Input:  {'filename': 'BridgeAmerNorth', 'filetype': 'mp4', 'url': 'https://www.youtube.com/watch?v=mp3RS0y77tY'}
out_filename_all BridgeAmerNorth.250714.1648.mp4
Working: manually quitting

2025-07-14 16:48:32.719  video capture and output file done
Created pickle file:  BridgeAmerNorth.250714.1648.pkl
output_file BridgeAmerNorth.250714.1648 mp4


In [13]:
print(output_file)

BridgeAmerNorth.250714.1648


In [14]:
# TEST: load data frame from the last Pickled output file
pickle_file = output_file
loaded_df = pd.read_pickle(pickle_file + ".pkl")
print("Timestamps Dataframe from Pickle file: ", pickle_file + ".pkl")
loaded_df.head()


Timestamps Dataframe from Pickle file:  BridgeAmerNorth.250714.1648.pkl


,0,1,2
0,0,1.752526e+09,2025-07-14 16:48:14.797
1,5,1.752526e+09,2025-07-14 16:48:14.913
2,10,1.752526e+09,2025-07-14 16:48:15.036
3,15,1.752526e+09,2025-07-14 16:48:15.148
4,20,1.752526e+09,2025-07-14 16:48:15.244


In [24]:
# BRIDGE OF THE AMERICAS NORTH - process video footage and detect and track vehicles
def obj_detection(video_filename, file_suffix="mp4", ShowEvents=True, ShowFrame=True):

    # Load the YOLO model
    model = YOLO('yolo11s.pt')
    #model = YOLO('yolo11m.pt')
    #model = YOLO('yolo11n.pt', verbose = False)  # this doesn't suppress the output I want supporessed

    class_list = model.names 
    #class_list

    video_file = video_filename + "." + file_suffix
    print("input video file: ", video_file)
    
    timestamps = pd.read_pickle(video_filename + ".pkl")
    print("Timestamps Dataframe from Pickle file: ", video_filename + ".pkl")
    #print(timestamps.head())   # for testing
    
    out_file = video_filename + "_proc." + file_suffix
        
    # Open the video file
    cap = cv2.VideoCapture(video_file)

    w, h, fps = (int(cap.get(x)) for x in (cv2.CAP_PROP_FRAME_WIDTH, cv2.CAP_PROP_FRAME_HEIGHT, cv2.CAP_PROP_FPS))
    video_writer = cv2.VideoWriter(out_file, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

    # color codes
    white = (255,255,255)
    red = (0,0,255)
    green = (0, 255, 0)
    yellow = (0,255,255)

    color_1 = (0,0,255) 
    color_2 = (255,255,0)
    
    #y and x positions of line
    # line 1 = A
    # line 2 = B
    line1_y_south = 670  # y position of line going south
    line1_x_south = 450  # x position of line going south
    line1_len_south = 520  # length of line 1 south
    
    line2_y_south = 840  # y position of line going south
    line2_x_south = 80  # x position of line going south    
    line2_len_south = 780  # length of line 1 south

    line1_y_north = 840  # y position of line going north
    line1_x_north = 1100  # x position of line going north
    line1_len_north = 800  # length of line 1 south
    
    line2_y_north = 670  # y position of line going north
    line2_x_north = 1050  # x position of line going north
    line2_len_north = 550  # length of line 2 north


    # Dictionary to store object counts by class
    class_counts_south1 = defaultdict(int)
    class_counts_south2 = defaultdict(int)
    class_counts_north1 = defaultdict(int)
    class_counts_north2 = defaultdict(int)
    
    # Dictionary to keep track of object IDs that have crossed the southbound line
    crossed_south_ids1 = set()
    crossed_south_ids2 = set()

    # Dictionary to keep track of object IDs that have crossed the northbound line
    crossed_north_ids1 = set()
    crossed_north_ids2 = set()

    track_id_prev = defaultdict(int)  # previous positions
    track_id_curr = defaultdict(int)  # current positions
    
    # Dictionary to track each vehicle crossing point A and point B and calculate time between
    veh_crossing = {}
    
    frame_ctr = 0
    event_ctr = 0
    event_tracker = []
    
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            #print("Not able to cap.read")
            break

        # Run YOLO tracking on the frame
        results = model.track(frame, persist=True, classes = [1,2,3,5,6,7], verbose=False) 
        # verbose = False - suppresses the console output
        #print(results)   # for testing


        # Ensure results are not empty
        if results[0].boxes.data is not None:
            
            #print("FRAME CTR: ", frame_ctr)   # print current frame number                    
            # Get the detected boxes, their class indices, and track IDs
            boxes = results[0].boxes.xyxy.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            class_indices = results[0].boxes.cls.int().cpu().tolist()
            confidences = results[0].boxes.conf.cpu()
            
            # line to track southbound
            cv2.putText(frame, 'Southbound Line A', (line1_x_south, line1_y_south - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_1, 1, cv2.LINE_AA)        
            cv2.line(frame, (line1_x_south , line1_y_south), 
                     (line1_x_south + line1_len_south, line1_y_south), color_1, 2)
            
            cv2.putText(frame, 'Southbound Line B', (line2_x_south, line2_y_south - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_1, 1, cv2.LINE_AA)        
            cv2.line(frame, (line2_x_south, line2_y_south), 
                     (line2_x_south + line2_len_south, line2_y_south), color_1, 2)

            # line 1 to track northbound
            cv2.putText(frame, 'Northbound Line A', (line1_x_north, line1_y_north - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_2, 1, cv2.LINE_AA)        
            cv2.line(frame, (line1_x_north, line1_y_north), 
                     (line1_x_north + line1_len_north, line1_y_north), color_2, 2)
            
            # line 2 to track northbound
            cv2.putText(frame, 'Northbound Line B', (line2_x_north, line2_y_north - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color_2, 1, cv2.LINE_AA)        
            cv2.line(frame, (line2_x_north, line2_y_north), 
                     (line2_x_north + line2_len_north, line2_y_north), color_2, 2)
            
            # Loop through each detected object
            for box, track_id, class_idx, conf in zip(boxes, track_ids, class_indices, confidences):
                x1, y1, x2, y2 = map(int, box)
                cx = (x1 + x2) // 2  # Calculate the center point
                cy = (y1 + y2) // 2            

                
                # if the object does not exist, create a new dictionary for vehicle crossing 
                # otherwise get the current dictionary
                
                if track_id in veh_crossing:
                    curr_veh_crossing = veh_crossing[track_id]
                else:
                    veh_crossing[track_id] =  {'DirA': None, 'A': None, 'Astr': None, 'DirB': None, 'B': None, 'Bstr': None, 'TimeBtwn': None}
                    curr_veh_crossing = veh_crossing[track_id]
                
                
                # store the current position of each object
                track_id_curr[track_id] = (cx, cy)
                class_name = class_list[class_idx]

                # center dot inside object
                cv2.circle(frame, (cx, cy), 4, red, -1)
            
                # id of object
                cv2.putText(frame, f"ID: {track_id} {class_name}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, yellow, 2)
                # frame around object
                cv2.rectangle(frame, (x1, y1), (x2, y2), green, 2) 

                # check if object ID already exists so there is previous position
                if frame_ctr > 0 and track_id in track_id_prev:

                    #timestamp = timestamps.iloc[frame_ctr,0 ] 
                    timestamp = timestamps.iloc[frame_ctr][1]  #get the datetime object 
                    
                    #timestamp = timestamps[frame_ctr]
                    datetime_object = datetime.fromtimestamp(timestamp)
                    #datetime_string = datetime_object.strftime("%Y-%m-%d %H:%M:%S")
                    datetime_string = datetime_object.strftime("%F %T.%f")[:-3]    

                    
                    # Check if the object has crossed the southbound line 1
                    if (cy > line1_y_south and track_id_prev[track_id][1] <= line1_y_south 
                        and cx >= line1_x_south and cx <= line1_x_south + line1_len_south  # x has to be within the line
                        and track_id not in crossed_south_ids1):
                        # Mark the object as crossed
                        crossed_south_ids1.add(track_id)
                        class_counts_south1[class_name] += 1
                        
                        event_ctr +=1
                        #new_event = (event_ctr, 'S1',track_id, class_name, timestamp, datetime_string)
                        new_event = [event_ctr, 'S1',track_id, class_name, timestamp, datetime_string]

                        curr_veh_crossing['DirA']='S'
                        curr_veh_crossing['A'] = timestamp
                        curr_veh_crossing['Astr'] = datetime_string
                        
                        event_tracker.append(new_event)

                        if ShowEvents:
                            print("new event: ", new_event)
                            print("veh crossing: ", curr_veh_crossing)


                    # Check if the object has crossed the southbound line 2
                    if (cy > line2_y_south and track_id_prev[track_id][1] <= line2_y_south 
                        and cx >= line2_x_south and cx <= line2_x_south + line2_len_south  # x has to be within the line
                        and track_id not in crossed_south_ids2):
                        # Mark the object as crossed
                        crossed_south_ids2.add(track_id)
                        class_counts_south2[class_name] += 1
                        
                        event_ctr +=1
                        #new_event = (event_ctr, 'S2',track_id, class_name, timestamp, datetime_string)
                        new_event = [event_ctr, 'S2',track_id, class_name, timestamp, datetime_string]
                        event_tracker.append(new_event)
                        
                        curr_veh_crossing['DirB']='S'
                        curr_veh_crossing['B'] = timestamp
                        curr_veh_crossing['Bstr'] = datetime_string
                        
                        if curr_veh_crossing['A'] is not None:
                            curr_veh_crossing['TimeBtwn'] = curr_veh_crossing['B'] - curr_veh_crossing['A']

                        if ShowEvents:
                            print("new event: ", new_event)
                            print("veh crossing: ", curr_veh_crossing)

                        
                                                
                    # Check if the object has crossed the northbound line 1
                    if (cy < line1_y_north and track_id_prev[track_id][1] >= line1_y_north 
                        and cx >= line1_x_north and cx <= line1_x_north + line1_len_north  # x has to be within the line
                        and track_id not in crossed_north_ids1):
                        # Mark the object as crossed
                        crossed_north_ids1.add(track_id)
                        class_counts_north1[class_name] += 1
                        
                        event_ctr +=1
                        #new_event = (event_ctr, 'N1',track_id, class_name, timestamp, datetime_string)
                        new_event = [event_ctr, 'N1',track_id, class_name, timestamp, datetime_string]
                        event_tracker.append(new_event)
                        
                            
                        curr_veh_crossing['DirA']='N'
                        curr_veh_crossing['A'] = timestamp
                        curr_veh_crossing['Astr'] = datetime_string
                            
                        if ShowEvents:
                            print("new event: ", new_event)
                            print("veh crossing: ", curr_veh_crossing)
                            
                            
                    # Check if the object has crossed the northbound line 2
                    if (cy < line2_y_north and track_id_prev[track_id][1] >= line2_y_north 
                        and cx >= line2_x_north and cx <= line2_x_north + line2_len_north  # x has to be within the line
                        and track_id not in crossed_north_ids2):
                        # Mark the object as crossed
                        crossed_north_ids2.add(track_id)
                        class_counts_north2[class_name] += 1

                        event_ctr +=1                        
                        #new_event = (event_ctr, 'N2',track_id, class_name, timestamp, datetime_string)
                        new_event = [event_ctr, 'N2',track_id, class_name, timestamp, datetime_string]
                        event_tracker.append(new_event)
                        
                        curr_veh_crossing['DirB']='N'
                        curr_veh_crossing['B'] = timestamp
                        curr_veh_crossing['Bstr'] = datetime_string
                        
                        if curr_veh_crossing['A'] is not None:
                            curr_veh_crossing['TimeBtwn'] = curr_veh_crossing['B'] - curr_veh_crossing['A']
                            
                        if ShowEvents:
                            print("new event: ", new_event)
                            print("veh crossing: ", curr_veh_crossing)
                        
            #print("PREV:")
            #print(track_id_prev)
            #print("CURR:")
            #print(track_id_curr)

            # store position detected object as previous position
            for track_id in track_ids:
                track_id_prev[track_id] = track_id_curr[track_id]
                                                            
            # Display the counts on the frame
            y_offset = 150
            
            cv2.putText(frame, "Southbound A vehicles" , (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_1, 2)
            y_offset += 30
            for class_name, count in class_counts_south1.items():
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_1, 2)
                y_offset += 30

                
            cv2.putText(frame, "Southbound B vehicles" , (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_1, 2)
            y_offset += 30
            for class_name, count in class_counts_south2.items():
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_1, 2)
                y_offset += 30

                
            y_offset += 100
            cv2.putText(frame, "Northbound A vehicles" , (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_2, 2)
            y_offset += 30
            for class_name, count in class_counts_north1.items():
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_2, 2)
                y_offset += 30

            cv2.putText(frame, "Northbound B vehicles" , (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_2, 2)
            y_offset += 30
            for class_name, count in class_counts_north2.items():
                cv2.putText(frame, f"{class_name}: {count}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color_2, 2)
                y_offset += 30

            
            # put the frame counter in video
            cv2.putText(frame, f"Frame: {frame_ctr}", (50, y_offset),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, green, 2)
            y_offset += 30
                
        # Show the frame
        if ShowFrame:
            cv2.namedWindow("YOLO Object Tracking & Counting", cv2.WINDOW_NORMAL)
            cv2.resizeWindow("YOLO Object Tracking & Counting", 1300, 900)
            cv2.imshow("YOLO Object Tracking & Counting", frame)    

        video_writer.write(frame)  # write the original frame

        
        # Exit loop if 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        
        frame_ctr += 1   # increment frame counter
            
    # Release resources
    cap.release()
    video_writer.release()
    
    cv2.destroyAllWindows()
    print("Output processed file: ", out_file)
    
    
    
    # convert event_tracking to DataFrame and write to pickle file
    event_tracker_df = pd.DataFrame(event_tracker, columns=['Event_ID', 'Event_Type', 'Obj_ID','Obj_Class','DT','DTStr'])    
    out_et_file = video_filename + "_proc_et.pkl" 
    event_tracker_df.to_pickle(out_et_file)
    print("Created pickle file for event tracker: ", out_et_file)
            
    # convert veh_crossing to DataFrame and write to pickle file
    veh_crossing_df = pd.DataFrame.from_dict(veh_crossing, orient='index')
    out_vc_file = video_filename + "_proc_vc.pkl" 
    veh_crossing_df.to_pickle(out_vc_file)
    print("Created pickle file for veh crossing: ", out_vc_file)
    
    # save to CSV files
    event_tracker_df.to_csv(video_filename + "_proc_et_df.csv", index=True)
    veh_crossing_df.to_csv(video_filename + "_proc_vc_df.csv", index=True)
    
    # save to Excel files
    event_tracker_df.to_excel(video_filename + "_proc_et_df.xlsx")
    veh_crossing_df.to_excel(video_filename + "_proc_vc_df.xlsx", index_label="Id")
    
    print("FINISHED")
    return event_tracker_df, veh_crossing_df
    

In [ ]:
# check our latest file
print(output_filetype)
print(output_file)

In [25]:
# process the last output file
et, vc = obj_detection(video_filename = output_file, file_suffix = output_filetype, ShowEvents=False, ShowFrame=False)



input video file:  BridgeAmerNorth.250714.1648.mp4
Timestamps Dataframe from Pickle file:  BridgeAmerNorth.250714.1648.pkl
Output processed file:  BridgeAmerNorth.250714.1648_proc.mp4
Created pickle file for event tracker:  BridgeAmerNorth.250714.1648_proc_et.pkl
Created pickle file for veh crossing:  BridgeAmerNorth.250714.1648_proc_vc.pkl
FINISHED


In [26]:
# processs SHORT FILE: BridgeAmerNorth.250712.1837
filetype = "mp4"
video_input = "BridgeAmerNorth.250712.1837"

print("video_input: ", video_input + "." + filetype)
et, vc = obj_detection(video_filename = video_input, file_suffix = filetype, ShowEvents=True, ShowFrame=True)
#video_input = "BridgeAmerNorth.250711.0818.mp4"

video_input:  BridgeAmerNorth.250712.1837.mp4
input video file:  BridgeAmerNorth.250712.1837.mp4
Timestamps Dataframe from Pickle file:  BridgeAmerNorth.250712.1837.pkl
new event:  [1, 'S1', 1, 'car', np.float64(1752359829.2541983), '2025-07-12 18:37:09.254']
veh crossing:  {'DirA': 'S', 'A': np.float64(1752359829.2541983), 'Astr': '2025-07-12 18:37:09.254', 'DirB': None, 'B': None, 'Bstr': None, 'TimeBtwn': None}
new event:  [2, 'S1', 9, 'car', np.float64(1752359829.7199693), '2025-07-12 18:37:09.719']
veh crossing:  {'DirA': 'S', 'A': np.float64(1752359829.7199693), 'Astr': '2025-07-12 18:37:09.719', 'DirB': None, 'B': None, 'Bstr': None, 'TimeBtwn': None}
new event:  [3, 'S2', 1, 'car', np.float64(1752359830.8723357), '2025-07-12 18:37:10.872']
veh crossing:  {'DirA': 'S', 'A': np.float64(1752359829.2541983), 'Astr': '2025-07-12 18:37:09.254', 'DirB': 'S', 'B': np.float64(1752359830.8723357), 'Bstr': '2025-07-12 18:37:10.872', 'TimeBtwn': np.float64(1.6181373596191406)}
new event:  